## Import libraries

In [2]:
import cassandra
from cassandra.cluster import Cluster

## Create a connection to the database

In [3]:
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

## Create a keyspace to do the work in and connect to it

In [4]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_keyspace
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
    session.set_keyspace("udacity_keyspace")
except Exception as e:
    print(e)

## Album library

We need to create a new music library of albums.  

Here is the information asked of the data:  
Query: Give me all the information from the music library about a given album  .
`select * from album_library WHERE album_name="Close To You"`  

Here is the collection of data:
![music_library_cassandra_db](images/music_library_cassandra_db_3.png)

**How should we model this data? What should be our Primary Key and Partition Key?**

## Data modeling

In [5]:
# Create table
create_album_library = """
CREATE TABLE IF NOT EXISTS album_library (
    album_name TEXT,
    artist_name TEXT,
    year INT,
    city TEXT,
    PRIMARY KEY (album_name, artist_name)
);
"""

try:
    session.execute(create_album_library)
except Exception as e:
    print(e)
    
# Insert data
insert_album_library = """
INSERT INTO album_library (album_name, artist_name, year, city)
VALUES (%s, %s, %s, %s);
"""

album_library_data = [
    ("Let it Be", "The Beatles", 1970, "Liverpool"),
    ("Rubber Soul", "The Beatles", 1965, "Oxford"),
    ("Beatles For Sale", "The Beatles", 1964, "London"),
    ("The Monkees", "The Monkees", 1966, "Los Angeles"),
    ("Close To You", "The Carpenters", 1970, "San Diego")
]

try:
    for item in album_library_data:
        session.execute(insert_album_library, item)
except Exception as e:
    print(e)

### Validate our data model

In [6]:
query = """
SELECT *
FROM album_library
WHERE album_name ='Close To You';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.album_name, row.artist_name, row.year, row.city)

Close To You The Carpenters 1970 San Diego


## Drop the table

In [7]:
try:
    session.execute("DROP TABLE album_library;")
except Exception as e:
    print(e)

## Close session and cluster connection

In [8]:
session.shutdown()
cluster.shutdown()